In [147]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely import geometry, ops
import folium

In [173]:
# Change the path for your own shapefile

fp = '/Users/santiagotoso/GoogleDrive/Master/Python/Merge multiple features into one/YELLOW_LINE/YELLOW_LINE.shp'
gdf = gpd.read_file(fp)
gdf.head()

,OBJECTID,ROADTYPE,ROADNUMBER,SPEEDLIMIT,LSN,ALSN,SEGLENGTH,MAINT_DATE,LCLSN,ROUTE,GlobalID,ShapeSTLen,geometry
0,1,M,2034,25.0,N MONROE DR,None,884.306000,1900-01-01T00:00:00.000Z,N Monroe Dr,YELLOW LINE,{A4118396-798E-4696-8A96-840B1B5D2755},1878.906347,LINESTRING (1566560.085239157 622017.450942233...
1,2,M,1894,25.0,WILSON DR,None,462.532000,1900-01-01T00:00:00.000Z,Wilson Dr,YELLOW LINE,{B5DBAB49-6E26-413C-8D31-EDCE79B7592E},1997.523488,LINESTRING (1565446.727885827 621462.594440400...
2,3,C,511,25.0,W CHURCH ST,None,22.047000,1900-01-01T00:00:00.000Z,W Church St,YELLOW LINE,{302C7D12-FE2D-410B-B98C-8C70C7F5C27B},1821.658116,LINESTRING (1564608.683630168 617115.379381567...
3,4,P,None,NaN,XENIA TOWNE SQ LOOP,None,1845.410257,2011-10-25T00:00:00.000Z,Xenia Towne Sq,YELLOW LINE,{C58CD268-B736-49E8-A117-637A2D447B15},1729.342722,LINESTRING (1565791.919780746 616837.511498987...
4,5,C,17,25.0,DAYTON ST,None,1073.247105,2010-12-16T00:00:00.000Z,Dayton St,YELLOW LINE,{BFA989AD-E546-4A1D-999C-889EC0596321},6104.191657,LINESTRING (1572984.187876493 659687.160707607...


In [174]:
gdf = gdf.to_crs({'init': 'epsg:4326'})

multitype = 'MultiLineString'
shapefile = gpd.GeoDataFrame()

def all_linestring(df):
    shape = gpd.GeoDataFrame()
    
    for _, row in df.iterrows():
        if row['geometry'].geom_type == multitype:
            multiline = row['geometry']
            merged = ops.linemerge(multiline)
            
            startstop = []
            for line in multitline:
                coords = [c for c in line.coords]
                startstop.append((coords[0],coords[:-1]))
            
            collection = ops.split(merged, geometry.MultiPoint(startstop))
            df2 = gpd.GeoDataFrame(geometry=[line for line in collection])
        
            shape = shape.append(df2)
        else: 
            line_series = gpd.GeoSeries({'geometry': row['geometry']})
            df2 = gpd.GeoDataFrame(data=line_series).T
            shape = shape.append(df2)
    return shape
    
all_li = all_linestring(gdf)

In [175]:
# Will create merge the rows of the shapefile into one feature

# Create an empty GeoDataFrame
shapefile = gpd.GeoDataFrame()
                
# Combine them into a multi-linestring
multi_line = geometry.MultiLineString(list(all_li.geometry))
                
# you can now merge the lines into one linestring (only in case the align)
merged_line = ops.linemerge(multi_line)
                
# create the geoseries
line_series = gpd.GeoSeries({'geometry': merged_line})
                
# append this shape to the shapefile
shapefile = shapefile.append(gpd.GeoDataFrame(data=line_series).T)
shapefile

,geometry
0,MULTILINESTRING ((-84.02143584607306 39.789862...


In [176]:
x = shapefile.bounds.loc[0,['minx', 'maxx']].mean()
y = shapefile.bounds.loc[0,['miny', 'maxy']].mean()

In [177]:
m = folium.Map(location = [y, x], tiles='cartodbpositron', zoom_start=13)

folium.GeoJson(shapefile.__geo_interface__).add_to(m)

m

In [178]:
prj = 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]]'

shapefile.to_file(driver = 'ESRI Shapefile',
                filename = '/Users/santiagotoso/GoogleDrive/Master/Python/Merge multiple features into one/outputs/yellow Line.shp',
                encoding='utf-8',
                crs_wkt=prj)